In [2]:
import spotipy

from spotipy.oauth2 import SpotifyClientCredentials

In [1]:
import pandas as pd
import requests

In [ ]:
# Coloca tus credenciales aquí
CLIENT_ID = ''
CLIENT_SECRET = ''
# Autenticación
auth_manager = SpotifyClientCredentials(client_id=CLIENT_ID, client_secret=CLIENT_SECRET)
sp = spotipy.Spotify(auth_manager=auth_manager)

In [ ]:
# Coloca tus credenciales de lastfm aquí
SHARED_SECRET = ''
API_KEY = ''

last_fm_connect = requests.get(f"http://www.last.fm/api/auth/?api_key={API_KEY}")

In [22]:
def extraccion_por_paises(diccionario_zonas, años):
    todos_los_datos = [] # Esto es una lista de dicts donde se van guardando los diccionarios de cada país
    for pais in diccionario_zonas.keys():
        datos_rango = sp.search(q = f'year:{años}', limit=50, offset=0, type='track', market={pais}) # Cada una tiene que cambiar en q el rango de años
        datos_por_pais = {}
        datos_por_pais['pais'] = pais
        datos_por_pais['datos'] = datos_rango
        todos_los_datos.append(datos_por_pais)
    return todos_los_datos
   

In [6]:
def filtro_de_datos(lista_datos_zonas, datos):
    for datos_pais in lista_datos_zonas:
        for element in datos_pais['datos']['tracks']['items']:
            datos['pais'].append(datos_pais['pais'])
            datos['popularity'].append(element['popularity'])
            datos['nombre'].append(element['name'])
            artistas = []
            for artista in element['artists']:
                artistas.append(artista['name'])   
            artistas_str = ', '.join(artistas)
            datos['artista'].append(artistas_str)

In [7]:
paises_este_europa = {
    "BY": "Bielorrusia",
    "BG": "Bulgaria",
    "CZ": "Chequia",
    "SK": "Eslovaquia",
    "HU": "Hungría",
    "MD": "Moldavia",
    "PL": "Polonia",
    "RO": "Rumanía",
    "UA": "Ucrania"}

In [23]:
p_este = extraccion_por_paises(paises_este_europa, '2000-2004')

In [24]:
dic_este = {'nombre': [], 'popularity': [], 'artista': [], 'pais': [], 'genero': []}

In [35]:
len(dic_este['genero'])

0

In [25]:
filtro_de_datos(p_este, dic_este)

In [26]:
dic_este

{'nombre': ['Сукины дети',
  'Angel',
  'Sonne',
  'Cabelo Raspadinho',
  'Without Me',
  'Devolva-Me',
  'ИСКАЛА',
  'Dar A Volta No Brasil - Ao Vivo',
  'Superman',
  'Csak Játék',
  'Numb',
  'Tarantula',
  'Mockingbird',
  "Guilty Conscience - Director's Cut",
  'ХОЧЕШЬ?',
  'Kaleidoscope World',
  'In the End',
  'Esperanzas',
  'Faint',
  'Spaceship',
  'Районы-кварталы',
  'Maçã - Ao Vivo',
  'Bring Me To Life',
  "'03 Bonnie & Clyde",
  'Toxicity',
  'Who Gives A F**K Where You From',
  'The Real Slim Shady',
  'St. Anger',
  'I Hate Everything About You',
  'Jenny from the Block',
  'Проклятый старый дом',
  'Un Condenado Te Amo',
  "Rollin' (Air Raid Vehicle)",
  'The Way I Am',
  'П.М.М.Л.',
  'Rama Hare - From "Soothradharan"',
  'Chop Suey!',
  'Oceano - Ao Vivo',
  'Наши юные смешные голоса',
  'Tvou Vuni Stale Znam',
  'Полковнику никто не пишет',
  'Somos Poucos Mas Somos Loucos',
  'Будь моим смыслом',
  'New Bibi-Hendl - Rap',
  'One Step Closer',
  'Zaluje',
  'Полов

In [36]:
lista_nombres_este = list(dic_este.values())[0]
lista_artistas_este = list(dic_este.values())[2]

In [37]:
def llamada_lastfm(lista_nombres, lista_artistas,datos):
    for i in range(len(lista_nombres)):
        try:
            nombre = str(lista_nombres[i]).replace(' ', '+')
            artista = lista_artistas[i].replace(' ', '+').replace(',', '+&')
            llamada_genero = requests.get(f"https://ws.audioscrobbler.com/2.0/?method=track.gettoptags&artist={artista}&track={nombre}&api_key={API_KEY}&autocorrect=1&format=json")
            try:
                genero_cancion = llamada_genero.json()['toptags']['tag'][0]['name']
                datos['genero'].append(genero_cancion)
            except KeyError:
                datos['genero'].append("NULL")
                print(f"La canción {lista_nombres[i]} no está registrada en lastfm")
                
            except IndexError:
                datos['genero'].append("NULL")
                print(f"La canción {lista_nombres[i]} no tiene tags")
            
        except:
            datos['genero'].append("NULL")
            print(f"Hay problemas con la canción {lista_nombres[i]}")

In [38]:
llamada_lastfm(lista_nombres_este, lista_artistas_este, dic_este)

La canción Сукины дети no tiene tags
La canción Angel no está registrada en lastfm
La canción Dar A Volta No Brasil - Ao Vivo no tiene tags
Hay problemas con la canción Superman
La canción Guilty Conscience - Director's Cut no está registrada en lastfm
La canción Kaleidoscope World no tiene tags
La canción Spaceship no tiene tags
La canción Maçã - Ao Vivo no está registrada en lastfm
La canción '03 Bonnie & Clyde no está registrada en lastfm
La canción Who Gives A F**K Where You From no está registrada en lastfm
La canción Rama Hare - From "Soothradharan" no está registrada en lastfm
La canción Oceano - Ao Vivo no tiene tags
La canción Tvou Vuni Stale Znam no tiene tags
La canción New Bibi-Hendl - Rap no tiene tags
La canción Половинка no tiene tags
La canción Вечно молодой no tiene tags
La canción Angel no está registrada en lastfm
La canción Yeah! (feat. Lil Jon & Ludacris) no está registrada en lastfm
La canción Dímelo bajito no tiene tags
La canción Dar A Volta No Brasil - Ao Vivo 

In [39]:
df_este = pd.DataFrame(dic_este)

In [40]:
df_este.index.name="id"

In [41]:
df_este

,nombre,popularity,artista,pais,genero
id,,,,,
0,Сукины дети,52,1.Kla$,BY,NULL
1,Angel,28,"Shaggy, Rayvon",BY,NULL
2,Sonne,73,Rammstein,BY,industrial metal
3,Cabelo Raspadinho,2,Chiclete Com Banana,BY,brasil
4,Without Me,83,Eminem,BY,rap
...,...,...,...,...,...
445,The Way I Am,0,Eminem,UA,rap
446,Seven Nation Army,78,The White Stripes,UA,rock
447,"Rama Hare - From ""Soothradharan""",0,"Raveendran, Edappal Viswan, Gayathri Ashokan",UA,NULL


In [42]:
df_este.to_csv("paises_este_2000_2004.csv")